# 📄 PDF -> LLM -> JSON -> Vector Database
## Step-by-step: Extract -> Display -> Save -> Load to Vector DB

### 🎯 Workflow
1. **Extract**: LLM reads PDF images and extracts data
2. **Display**: Show extracted data on screen
3. **Save**: Store results as JSON file
4. **Load**: Import JSON data into Pinecone vector database

In [ ]:
!pip install langchain langchain-openai langchain-google-genai langchain-pinecone pinecone PyMuPDF pandas

In [1]:
import os
import json
import base64
from pathlib import Path
import getpass
from typing import List, Dict
import pandas as pd
from datetime import datetime

# PDF processing
import fitz  # PyMuPDF

# LangChain and OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document

# Google models
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_ollama import ChatOllama, OllamaEmbeddings

# Pinecone
from pinecone import Pinecone, ServerlessSpec

print("✅ All imports successful!")

d:\Projects\evidence-auditor\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projects\evidence-auditor\.venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


✅ All imports successful!


In [2]:
# Configuration
DATA_DIR = "data"
JSON_OUTPUT_FILE = "extracted_invoice_data.json"
INDEX_NAME = "invoice-json-vectors-ollama"

# API Keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass.getpass("OpenAI API Key: ")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") or getpass.getpass("Google API Key: ")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") or getpass.getpass("Pinecone API Key: ")


print("✅ Configuration set!")

✅ Configuration set!


## Step 1: Extract Data from PDFs using LLM

In [ ]:
# Initialize LLM for vision processing

# Ollama option (local):
llm_vision = ChatOllama(
    model="llama3.2:1b",
    temperature=0
)

def pdf_to_base64_images(pdf_path: str) -> List[str]:
    """Convert PDF pages to base64 images"""
    doc = fitz.open(pdf_path)
    base64_images = []
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
        img_data = pix.tobytes("png")
        img_base64 = base64.b64encode(img_data).decode()
        base64_images.append(img_base64)
    
    doc.close()
    return base64_images

def extract_invoice_data(image_base64: str, filename: str, page_num: int) -> Dict:
    """Extract structured data from invoice image using LLM"""
    prompt = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """Extract invoice information from this image:

1. vendor_name: Company issuing the invoice
2. invoice_number: Invoice ID/reference
3. date: Invoice date
4. amount: Total amount
5. full_text: All visible text

Return JSON: {"vendor_name": "...", "invoice_number": "...", "date": "...", "amount": "...", "full_text": "..."}
Use "NOT_FOUND" if field not visible."""
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{image_base64}"}
                }
            ]
        }
    ]
    
    try:
        response = llm_vision.invoke(prompt)
        content = response.content.strip()
        
        # Clean JSON
        if '```json' in content:
            content = content.split('```json')[1].split('```')[0].strip()
        elif '```' in content:
            content = content.split('```')[1].strip()
        
        data = json.loads(content)
        data.update({
            'filename': filename,
            'page_number': page_num,
            'extraction_timestamp': datetime.now().isoformat(),
            'extraction_method': 'ollama'
        })
        
        return data
        
    except Exception as e:
        return {
            'filename': filename,
            'page_number': page_num,
            'vendor_name': 'ERROR',
            'invoice_number': 'ERROR',
            'date': 'ERROR',
            'amount': 'ERROR',
            'full_text': f'Extraction failed: {str(e)}',
            'extraction_timestamp': datetime.now().isoformat(),
            'extraction_method': 'gpt4_vision_error'
        }

print("✅ LLM extraction functions ready!")

✅ LLM extraction functions ready!


In [16]:
# Process all PDFs and extract data
data_dir = Path(DATA_DIR)
pdf_files = list(data_dir.glob("invoice_*.pdf"))

print(f"📁 Found {len(pdf_files)} PDF files")
print("🔍 Starting LLM extraction...\n")

extracted_data = []

for pdf_file in sorted(pdf_files):
    print(f"📄 Processing: {pdf_file.name}")
    
    # Convert to images
    base64_images = pdf_to_base64_images(str(pdf_file))
    print(f"📸 Converted to {len(base64_images)} images")
    
    # Extract from each page
    for page_num, image_base64 in enumerate(base64_images, 1):
        print(f"  🔍 Extracting page {page_num}...")
        
        invoice_data = extract_invoice_data(image_base64, pdf_file.name, page_num)
        extracted_data.append(invoice_data)
        
        # Show extraction result
        vendor = invoice_data.get('vendor_name', 'N/A')[:20]
        invoice_num = invoice_data.get('invoice_number', 'N/A')
        print(f"    ✅ {vendor} | {invoice_num}")
    
    print()

print(f"✅ Extracted data from {len(extracted_data)} pages total")

📁 Found 6 PDF files
🔍 Starting LLM extraction...

📄 Processing: invoice_1.pdf
📸 Converted to 1 images
  🔍 Extracting page 1...
    ✅ ERROR | ERROR

📄 Processing: invoice_2.pdf
📸 Converted to 1 images
  🔍 Extracting page 1...
    ✅ ERROR | ERROR

📄 Processing: invoice_3.pdf
📸 Converted to 1 images
  🔍 Extracting page 1...
    ✅ ERROR | ERROR

📄 Processing: invoice_4.pdf
📸 Converted to 1 images
  🔍 Extracting page 1...
    ✅ ERROR | ERROR

📄 Processing: invoice_5.pdf
📸 Converted to 1 images
  🔍 Extracting page 1...
    ✅ ERROR | ERROR

📄 Processing: invoice_6.pdf
📸 Converted to 1 images
  🔍 Extracting page 1...
    ✅ ERROR | ERROR

✅ Extracted data from 6 pages total


## Step 2: Display Extracted Data

In [17]:
extracted_data

[{'filename': 'invoice_1.pdf',
  'page_number': 1,
  'vendor_name': 'ERROR',
  'invoice_number': 'ERROR',
  'date': 'ERROR',
  'amount': 'ERROR',
  'full_text': 'Extraction failed: Expecting value: line 1 column 1 (char 0)',
  'extraction_timestamp': '2025-10-09T08:53:24.730098',
  'extraction_method': 'gpt4_vision_error'},
 {'filename': 'invoice_2.pdf',
  'page_number': 1,
  'vendor_name': 'ERROR',
  'invoice_number': 'ERROR',
  'date': 'ERROR',
  'amount': 'ERROR',
  'full_text': 'Extraction failed: Expecting value: line 1 column 1 (char 0)',
  'extraction_timestamp': '2025-10-09T08:53:26.032359',
  'extraction_method': 'gpt4_vision_error'},
 {'filename': 'invoice_3.pdf',
  'page_number': 1,
  'vendor_name': 'ERROR',
  'invoice_number': 'ERROR',
  'date': 'ERROR',
  'amount': 'ERROR',
  'full_text': 'Extraction failed: Expecting value: line 1 column 1 (char 0)',
  'extraction_timestamp': '2025-10-09T08:53:27.307738',
  'extraction_method': 'gpt4_vision_error'},
 {'filename': 'invoice

In [18]:
# Display extracted data in a nice table
if extracted_data:
    df = pd.DataFrame(extracted_data)
    
    # Create display table
    display_df = pd.DataFrame({
        'Filename': df['filename'],
        'Page': df['page_number'],
        'Vendor Name': df['vendor_name'].str[:30],
        'Invoice Number': df['invoice_number'],
        'Date': df['date'],
        'Amount': df['amount'],
        'Status': df['extraction_method'].apply(lambda x: '✅' if 'error' not in x else '❌')
    })
    
    print("📊 EXTRACTED INVOICE DATA")
    print("=" * 80)
    print(display_df.to_string(index=False))
    
    # Summary statistics
    total = len(df)
    successful = len(df[df['vendor_name'] != 'ERROR'])
    
    print(f"\n📈 EXTRACTION SUMMARY:")
    print(f"Total pages processed: {total}")
    print(f"Successful extractions: {successful}/{total} ({successful/total*100:.1f}%)")
    
    # Show sample full text
    print(f"\n📝 SAMPLE EXTRACTED TEXT:")
    for i, row in df.head(2).iterrows():
        if row['full_text'] != 'ERROR' and 'NOT_FOUND' not in row['full_text']:
            print(f"\n{row['filename']} (Page {row['page_number']}):")
            print(f"{row['full_text'][:200]}...")
            break
else:
    print("❌ No data extracted")

📊 EXTRACTED INVOICE DATA
     Filename  Page Vendor Name Invoice Number  Date Amount Status
invoice_1.pdf     1       ERROR          ERROR ERROR  ERROR      ❌
invoice_2.pdf     1       ERROR          ERROR ERROR  ERROR      ❌
invoice_3.pdf     1       ERROR          ERROR ERROR  ERROR      ❌
invoice_4.pdf     1       ERROR          ERROR ERROR  ERROR      ❌
invoice_5.pdf     1       ERROR          ERROR ERROR  ERROR      ❌
invoice_6.pdf     1       ERROR          ERROR ERROR  ERROR      ❌

📈 EXTRACTION SUMMARY:
Total pages processed: 6
Successful extractions: 0/6 (0.0%)

📝 SAMPLE EXTRACTED TEXT:

invoice_1.pdf (Page 1):
Extraction failed: Expecting value: line 1 column 1 (char 0)...


## Step 3: Save Data as JSON File

In [19]:
# Save extracted data to JSON file
if extracted_data:
    # Add metadata to JSON
    json_output = {
        "metadata": {
            "extraction_date": datetime.now().isoformat(),
            "total_files_processed": len(pdf_files),
            "total_pages_extracted": len(extracted_data),
            "successful_extractions": len([d for d in extracted_data if d['vendor_name'] != 'ERROR']),
            "extraction_model": "gpt-4o-mini"
        },
        "extracted_data": extracted_data
    }
    
    # Save to file
    with open(JSON_OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(json_output, f, indent=2, ensure_ascii=False)
    
    file_size = Path(JSON_OUTPUT_FILE).stat().st_size / 1024  # KB
    
    print(f"💾 SAVED TO JSON FILE")
    print(f"File: {JSON_OUTPUT_FILE}")
    print(f"Size: {file_size:.1f} KB")
    print(f"Records: {len(extracted_data)}")
    
    # Show JSON structure
    print(f"\n📋 JSON STRUCTURE:")
    print(f"├── metadata (extraction info)")
    print(f"└── extracted_data ({len(extracted_data)} records)")
    print(f"    ├── filename, page_number")
    print(f"    ├── vendor_name, invoice_number, date, amount")
    print(f"    ├── full_text (complete extracted text)")
    print(f"    └── extraction_timestamp, extraction_method")
    
else:
    print("❌ No data to save")

💾 SAVED TO JSON FILE
File: extracted_invoice_data.json
Size: 2.5 KB
Records: 6

📋 JSON STRUCTURE:
├── metadata (extraction info)
└── extracted_data (6 records)
    ├── filename, page_number
    ├── vendor_name, invoice_number, date, amount
    ├── full_text (complete extracted text)
    └── extraction_timestamp, extraction_method


## Step 4: Load JSON Data into Vector Database

In [20]:
# Initialize Pinecone components

#Ollama options (local):
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large"
)

pc = Pinecone(api_key=PINECONE_API_KEY)

# Create index if needed
existing = [idx["name"] for idx in pc.list_indexes()]
if INDEX_NAME not in existing:
    print(f"Creating Pinecone index: {INDEX_NAME}")
    pc.create_index(
        name=INDEX_NAME,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    import time
    time.sleep(10)

index = pc.Index(INDEX_NAME)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

print("✅ Pinecone components initialized!")

Creating Pinecone index: invoice-json-vectors-ollama
✅ Pinecone components initialized!


In [21]:
# Load JSON file and convert to vector documents
print(f"📂 Loading data from {JSON_OUTPUT_FILE}...")

try:
    with open(JSON_OUTPUT_FILE, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    extracted_records = json_data['extracted_data']
    metadata_info = json_data['metadata']
    
    print(f"✅ Loaded {len(extracted_records)} records from JSON")
    print(f"📊 Extraction date: {metadata_info['extraction_date']}")
    
    # Convert to LangChain documents for vector storage
    documents = []
    
    for record in extracted_records:
        # Only create documents for successful extractions with text
        full_text = record.get('full_text', '')
        if full_text and full_text not in ['ERROR', 'NOT_FOUND'] and 'failed' not in full_text.lower():
            doc = Document(
                page_content=full_text,
                metadata={
                    'filename': record['filename'],
                    'page_number': record['page_number'],
                    'vendor_name': record['vendor_name'],
                    'invoice_number': record['invoice_number'],
                    'date': record['date'],
                    'amount': record['amount'],
                    'extraction_timestamp': record['extraction_timestamp'],
                    'extraction_method': record['extraction_method'],
                    'doc_type': 'invoice',
                    'source': 'json_file'
                }
            )
            documents.append(doc)
    
    print(f"📄 Created {len(documents)} documents for vector storage")
    
except FileNotFoundError:
    print(f"❌ JSON file {JSON_OUTPUT_FILE} not found. Run extraction steps first.")
    documents = []

📂 Loading data from extracted_invoice_data.json...
✅ Loaded 6 records from JSON
📊 Extraction date: 2025-10-09T08:54:53.995507
📄 Created 0 documents for vector storage


In [22]:
# Store documents in Pinecone vector database
if documents:
    print(f"🗄️ Storing {len(documents)} documents in Pinecone vector database...")
    
     # --- IGNORE ---
    try:
        vector_ids = vector_store.add_documents(documents)
        
        print(f"✅ Successfully stored {len(vector_ids)} vectors!")
        print(f"📊 Sample vector IDs: {vector_ids[:3]}...")
        
        # Verify storage
        stats = index.describe_index_stats()
        print(f"📈 Total vectors in index: {stats.total_vector_count}")
        print(f"🔢 Vector dimension: {stats.dimension}")
        
    except Exception as e:
        print(f"❌ Error storing in Pinecone: {e}")
else:
    print("❌ No documents to store in vector database")

❌ No documents to store in vector database


## Step 5: Test Vector Search

In [23]:
# Test vector search on the loaded data
print("🔍 Testing vector search on JSON-loaded data:")
print("=" * 50)

test_queries = [
    "invoice amount payment",
    "vendor company name",
    "total cost price"
]

for query in test_queries:
    print(f"\n🔍 Query: '{query}'")
    
    try:
        results = vector_store.similarity_search(query, k=3)
        
        if results:
            for i, doc in enumerate(results, 1):
                filename = doc.metadata.get('filename', 'Unknown')
                vendor = doc.metadata.get('vendor_name', 'Unknown')
                invoice_num = doc.metadata.get('invoice_number', 'Unknown')
                source = doc.metadata.get('source', 'Unknown')
                
                print(f"  {i}. {filename} - {vendor} ({invoice_num}) [from {source}]")
                print(f"     Content: {doc.page_content[:100]}...")
        else:
            print("  No results found")
            
    except Exception as e:
        print(f"  ❌ Search error: {e}")

print("\n✅ Complete workflow: PDF -> LLM -> JSON -> Vector DB finished!")

🔍 Testing vector search on JSON-loaded data:

🔍 Query: 'invoice amount payment'
  1. invoice_2.pdf - reallygreatsite.com (NOT_FOUND) [from json_file]
     Content: INVOICE
Date:
Bill to:
No. Invoice :
123 Anywhere St.,Any City, ST
12345
Date
Item Description
Price...
  2. invoice_6.pdf - Wardiere Inc (1009-01) [from json_file]
     Content: NO. 1009-01
DUE 1 APRIL 2022
INVOICE
INVOICE TO
Bailey Dupont
Studio Shadowe
123 Anywhere St.,
Any C...
  3. invoice_3.pdf - AVERY DAVIS Business Consultant (NOT_FOUND) [from json_file]
     Content: AVERY DAVIS
Business Consultant
123 Anywhere St.,
Any City, ST 12345
hello@reallygreatsite.com
123-4...

🔍 Query: 'vendor company name'
  1. invoice_2.pdf - reallygreatsite.com (NOT_FOUND) [from json_file]
     Content: INVOICE
Date:
Bill to:
No. Invoice :
123 Anywhere St.,Any City, ST
12345
Date
Item Description
Price...
  2. invoice_6.pdf - Wardiere Inc (1009-01) [from json_file]
     Content: NO. 1009-01
DUE 1 APRIL 2022
INVOICE
INVOICE TO
Bailey Dup